


# Tutorial: Getting the Percent Distribution of Benthic and Geomorphic Classes Given an EEZ

### Purpose
In this tutorial, we'll walk through how to get the percent distribution of benthic and geomorphic classes within the bounds of an Exclusive Economic Zone (EEZ). The percent distribution is based off of the Allen Coral Atlas (ACA) and the EEZ is based off of the Marine Regions web map services layer. This is a starting guide that can serve as a launchpad for further analyses such as comparing the percent distribution of multiple countries and integration of other datasets for analysis.


### Setup
We recommend using a Conda environment with the packages listed below. Instructions to add a Conda environment to a jupyter notebook can be found [here](https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084), and Conda documentation can be found [here](https://docs.conda.io/en/latest/).
 
 ### Contents
 
 1. Creating a Dictionary of Countries/Areas and Their Associated Marine Regions ID (MRGID)

 2. Retrieving Benthic and Geomorphic Layers for a Given EEZ

 3. Reproject the Layers to an Equal Area Projection
 
 4. Calculate Percent Distribution for the Benthic and Geomorphic Classes
 
 5. Visualization: Reproducing Donut Charts from the Web Platform
 
 6. Comparison between Tonga and Jamaica
 

### Considerations

- Currently, the Equal Area projection used in this tutorial is Eckert IV. Further work and research is being conducted to find the optimal projection that accurately reflects the total area for an EEZ.
- Error handling has not been implemented for clarity's sake
- Unit testing has not been implemented but is advised when doing these analyses
- For regions such as Fiji, it is advised to change projections to a Pacific centered projection


### Packages to Import
- Note: while no explicit calls are made, the `rtree` package is a required dependency for geopandas spatial joins

In [ ]:
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import rtree
import requests

from osgeo import ogr

from owslib.wfs import WebFeatureService
from owslib.wms import WebMapService

# 1. Creating a Dictionary of Territories and Their Associated Marine Regions ID (MRGID)

First, we'll create a dictionary with the keys being all the areas within the ACA and the values being the associated Marine Regions ID. We'll need the Marine Regions ID in the second section in order to obtain the EEZ of a specific country/area. To do so, we'll grab the Marine Regions EEZ map and restrict the output with a bounding box that is representative of the ACA area.

Note that this request can take up to 3 minutes.

In [ ]:
wfs = WebFeatureService(url='https://geo.vliz.be/geoserver/MarineRegions/wfs', version='1.1.0')
response = wfs.getfeature(typename='MarineRegions:eez', 
                          bbox=(-180,-23.5,180,20), 
                          srsname='urn:x-ogc:def:crs:EPSG:4326', 
                          outputFormat='application/json')
eez_data = json.load(response)

Here, we create the dictionary and ensure that disputed and/or shared territories are included with the following filtering parameters.

In [ ]:
mrgid_dict = {}
for country in eez_data['features']:
    if country['properties']['pol_type'] == '200NM':
        mrgid_dict[country['properties']['territory1']] = country['properties']['mrgid']
    else:
        mrgid_dict[country['properties']['geoname']] = country['properties']['mrgid']

Now that we have successfully created a dictionary of countries/areas with their associated MRGID, we can take a look at the first 10 entries.

In [ ]:
{eez: mrgid_dict[eez] for eez in list(mrgid_dict)[:10]}

In total, we have 171 entries.

In [ ]:
len(mrgid_dict)

# 2. Retrieving Benthic and Geomorphic Layers for a Given EEZ

With our dictionary completed, we can now go through the process of obtaining the benthic and geomorphic layers for a specific area of interest. There are four steps to this:

1. Getting the EEZ area based off of a MRGID
2. Making a bounding box from said EEZ area
3. Obtaining the layers from the ACA using the bounding box
4. Getting the intersection between the ACA and the Marine Regions EEZ area


Here is the overview of the helper functions to achieve these four steps:

- `get_eez_map(country_mrgid)`: Gets the EEZ geojson from the Marine Regions GeoServer
- `make_bounding_box(geojson)`: Transforms a geojson to a bounding box
- `get_aca_layer(layer, bounding_box)`: Given a bounding box, a request is made and downloads either the benthic and geomorphic layers from the Allen Coral Atlas
- `eez_intersection(aca_layer,eez)`: Get the intersection between the Allen Coral Atlas layers and the EEZ geojson from Marine Regions

Note: The external inquiry made in the `get_eez_map` method can take more than a minute.

In [ ]:
def get_eez_map(country_mrgid):
    '''Given a country mrgid(Marine region country code), returns a geojson of a country\'s EEZ'''
    url = 'https://geo.vliz.be/geoserver/MarineRegions/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=eez&cql_filter=mrgid=' + str(country_mrgid) + '&outputformat=application/json'
    response = requests.get(url)
    eez = response.json()
    return eez

The `make_bounding_box` helper function grabs the first feature and protects if from wrapping at the antimeridian.

In [ ]:
def make_bounding_box(geojson):
    '''From a geojson, creates a bounding box which is a list of 4 coordinates'''
    geom_text = json.dumps(geojson["features"][0]["geometry"])
    geom = ogr.CreateGeometryFromJson(geom_text)
    env = geom.GetEnvelope()

    MAX_LONG = 180.0
    MIN_LONG = -180.0
    MAX_LAT = 90.0
    MIN_LAT = -90.0

    return (max(min(MAX_LONG, env[0]), MIN_LONG),
            max(min(MAX_LAT, env[2]), MIN_LAT),
            max(min(MAX_LONG, env[1]), MIN_LONG),
            max(min(MAX_LAT, env[3]), MIN_LAT))

In [ ]:
def get_aca_layer(layer, bounding_box):
    '''Gets either geomorphic or benthic layer in geojson from the ACA'''
    # layer options: ['reef_polygons_benthic_expanded'] or ['reef_polygons_geomorphic_expanded']
    wms = WebMapService('https://allencoralatlas.org/geoserver/coral-atlas/wms', version='1.3.0')
    country_aca_layer = wms.getmap(layers=layer, 
                                   srs='EPSG:4326', 
                                   bbox=bounding_box, 
                                   size=(1000,1000),
                                   format='application/json;type=geojson')
    try:
        country_aca_layer_geojson = json.load(country_aca_layer)
        return country_aca_layer_geojson
    except ValueError:
        if country_aca_layer._response.status_code == 429:
            print('Too many requests have been made in a given amount of time, please wait another minute to send another request.')
        else:
            print('Error acquiring layer, response code: ' + country_aca_layer._response.status_code)         

After obtaining the intersection, the layers are now in GeoDataFrame for ease of calculating percent distribution in the next steps.

In [ ]:
def eez_intersection(aca_layer, eez):
    '''After getting a benthic/geomorphic layer, get EEZ region in the form of a geodataframe'''
    aca_layer = gpd.GeoDataFrame.from_features(aca_layer)
    eez = gpd.GeoDataFrame.from_features(eez)
    eez_aca_combined = gpd.sjoin(aca_layer, eez)

    return eez_aca_combined

## Example: Obtaining the Benthic and Geomorphic Layers of Tonga
Let's take what we've learned so far to retrieve both the benthic and geomorphic layers of Tonga. We'll first use our MRGID dictionary created in the beginning of the tutorial, `mrgid_dict`, to find Tonga's MRGID.

In [ ]:
mrgid_dict['Tonga']

We'll now use this ID to get the bounding box of Tonga's EEZ.

In [ ]:
tonga = get_eez_map(mrgid_dict['Tonga'])
tongabbox = make_bounding_box(tonga)

Here, we can take a look at the bounding box coordinates.

In [ ]:
tongabbox

We'll now take our `tongabbox` and obtain the benthic and geomorphic layers using our previously defined `get_aca_layer` function we defined earlier. Currently, to request the benthic layer, ['benthic_data_verbose'], is used and to request the geomorphic layer, ['geomorphic_data_verbose'], is used.

Note: To prevent overloading the GeoServer, it is advised to wait a minute between each request.

In [ ]:
benthic_tonga = get_aca_layer(['benthic_data_verbose'], tongabbox)

In [ ]:
geomorphic_tonga = get_aca_layer(['geomorphic_data_verbose'], tongabbox)

We'll now obtain the intersection between the ACA layers and the Marine Regions EEZ using our defined `eez_intersection` function. The intersection will return a GeoDataFrame which will make data analysis easier.

In [ ]:
benthic_tonga_eez = eez_intersection(benthic_tonga, tonga)

In [ ]:
geomorphic_tonga_eez = eez_intersection(geomorphic_tonga, tonga)

Taking a closer look at what we have, we see that each row of the geodataframe is a polygon that is one of the benthic/geomorphic subtypes.

In [ ]:
benthic_tonga_eez

In [ ]:
geomorphic_tonga_eez

# 3. Reprojection of Benthic and Geomorphic Layers

We now have the benthic and geomorphic layers for Tonga, but before we can do any further analyses, we have to ensure that we have the right map projection. More information on choosing the correct map projections can be found [here](https://learn.arcgis.com/en/projects/choose-the-right-projection/). To get an accurate estimate of percent distribution, it is imperative to reproject the layers to an equal area projection. The ACA has a default projection of EPSG:4326. Within our `reproject` function below, we've chosen the Mollweide projection (ESRI:54009) which has a suitable extent for the world.

A quick overview of the `reproject` function is to set the original CRS before projecting, which in this case for the ACA is EPSG:4326, and then we reproject to our desired map projection and simultaneously calculate the area for each feature in km<sup>2</sup>. This new area is also added as a new column in our geodataframe.

In [ ]:
def reproject(eez_geodataframe, projection_identifier):
    '''Reprojects a geodataframe to the user provided projection'''
    # e.g. Mollweide would be 'ESRI:54009'
    # set original CRS before reprojecting, in this case, the data obtained 
    # from the Allen Coral Atlas is in the EPSG:4326 format
    eez_geodataframe.crs = 'EPSG:4326'
    
    # adds a column, 'reprojected_area' that converts the 
    # current area to user provided projection
    eez_geodataframe['reprojected_area'] = eez_geodataframe['geometry'].to_crs(projection_identifier).map(lambda p: p.area / 10**6)
    
    return eez_geodataframe

In [ ]:
reprojected_tonga_benthic = reproject(benthic_tonga_eez, 'ESRI:54009')

In [ ]:
reprojected_tonga_geomorphic = reproject(geomorphic_tonga_eez, 'ESRI:54009')

# 4. Calculating the Percent Distribution of Benthic and Geomorphic Classes

With our reprojected benthic and geomorphic layers, we can now calculate the percent distribution by benthic/geomorphic subtypes.

In the `calculate_percent_distribution` function, we first calculate the total area for the classes and then calculate the subtotals in a new geodataframe.

In [ ]:
def calculate_percent_distribution(eez_geodataframe): 
    # generates a new dataframe containing the subtotal 
    # of the area of benthic/geomorphic classes 
    eez_geodataframe_subtotals = eez_geodataframe.groupby('class_name')['reprojected_area'].sum().to_frame()
    
    # adds a new column that has the percent distribution
    # for each class
    eez_geodataframe_subtotals['percent_distribution'] = eez_geodataframe_subtotals['reprojected_area'].map(lambda p: p/sum(eez_geodataframe_subtotals['reprojected_area']) * 100)

    return eez_geodataframe_subtotals

### Tonga Benthic Class Percent Distribution
Let's now calculate the benthic percent distribution for Tonga.

In [ ]:
benthic_tonga = calculate_percent_distribution(reprojected_tonga_benthic)

Now, if we take a look at our newly generated dataframe, we can see the benthic composition of the EEZ of Tonga.

In [ ]:
benthic_tonga

### Tonga Geomorphic Class Percent Distribution
And we can do the same for the geomorphic class.

In [ ]:
geomorphic_tonga = calculate_percent_distribution(reprojected_tonga_geomorphic)

In [ ]:
geomorphic_tonga

# 5. Visualization: Reproducing Donut Charts from the Web Platform
Using the `matplotlib` library and this [tutorial](https://towardsdatascience.com/donut-plot-with-matplotlib-python-be3451f22704), here's how we can reproduce the graph from the web platform. In addition, the graphs can be easily customized as needed.

### Tonga Benthic Class Distribution

In [ ]:
def plot_benthic(benthic_layer_geodataframe, eez_name):
    # Donut chart
    labels = list(benthic_layer_geodataframe.index)
    color_dict = {'Coral/Algae': '#ff6157',
                  'Microalgal Mats': '#9bcc4f',
                  'Rock': '#b19c3a',
                  'Rubble': '#e0d05e',
                  'Sand': '#ffffbe',
                  'Seagrass': '#668438'}
    colors = []
    for class_name in list(benthic_layer_geodataframe.index):
        if color_dict[class_name]:
            colors.append(color_dict[class_name])
            
    sizes = list(benthic_layer_geodataframe['percent_distribution'])

    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90)

    # Draw circle
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    # Equal aspect ratio ensures that pie is drawn as a circle
    ax1.axis('equal')  
    plt.tight_layout()
    plt.title(eez_name + ' Benthic Class Distribution', pad=20)
    plt.show()

In [ ]:
plot_benthic(benthic_tonga, 'Tonga')

### Tonga Geomorphic Class Distribution

In [ ]:
def plot_geomorphic(geomorphic_layer_geodataframe, eez_name):
    # Donut chart
    labels = list(geomorphic_layer_geodataframe.index)
    colors = []
    color_dict = {'Back Reef Slope': '#befbff',
                 'Deep Lagoon': '#2ca2f9',
                 'Inner Reef Flat': '#c5a7cb',
                 'Outer Reef Flat': '#9a7fa4',
                 'Plateau': '#cd6812',
                 'Reef Crest': '#614272',
                 'Reef Slope': '#288471',
                 'Shallow Lagoon': '#77d0fc',
                 'Sheltered Reef Slope': '#10bda6',
                 'Terrestrial Reef Flat': '#fbdefb'}
    for class_name in list(geomorphic_layer_geodataframe.index):
        if color_dict[class_name]:
            colors.append(color_dict[class_name])
        
    sizes = list(geomorphic_layer_geodataframe['percent_distribution'])
    
    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90)

    # Draw circle
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    # Equal aspect ratio ensures that pie is drawn as a circle
    ax1.axis('equal')  
    plt.tight_layout()
    plt.title(eez_name + ' Geomorphic Class Distribution', pad=20)
    plt.show()

In [ ]:
plot_geomorphic(geomorphic_tonga, 'Tonga')

# 6. Comparison between Tonga and Jamaica
We can also easily see the diversity between Tonga and Jamaica. Briefly, we'll go through the steps as we did with Tonga for Jamaica.

In [ ]:
mrgid_dict['Jamaica']

In [ ]:
jamaica = get_eez_map(mrgid_dict['Jamaica'])
jamaica_bbox = make_bounding_box(jamaica)

In [ ]:
jamaica_bbox

In [ ]:
jamaica_geomorphic = get_aca_layer(['geomorphic_data_verbose'], jamaica_bbox)

In [ ]:
jamaica_benthic = get_aca_layer(['benthic_data_verbose'], jamaica_bbox)

In [ ]:
jamaica_geomorphic_eez = eez_intersection(jamaica_geomorphic, jamaica)

In [ ]:
jamaica_benthic_eez = eez_intersection(jamaica_benthic, jamaica)

In [ ]:
reprojected_jamaica_benthic = reproject(jamaica_benthic_eez, 'ESRI:54012')

In [ ]:
reprojected_jamaica_geomorphic = reproject(jamaica_geomorphic_eez, 'ESRI:54012')

In [ ]:
benthic_jamaica = calculate_percent_distribution(reprojected_jamaica_benthic)

In [ ]:
geomorphic_jamaica = calculate_percent_distribution(reprojected_jamaica_geomorphic)

Let's first take a look at the benthic distributions:

In [ ]:
plot_benthic(benthic_tonga, 'Tonga')
plot_benthic(benthic_jamaica, 'Jamaica')

With the charts displayed, we can appreciate the diversity between the corals in the Caribbean and South Pacific Ocean. Characteristic of the Caribbean environment, there is a high density of seagrass.
Let's also take a look at the geomorphic distribution:

In [ ]:
plot_geomorphic(geomorphic_tonga, 'Tonga')
plot_geomorphic(geomorphic_jamaica, 'Jamaica')